In [20]:
def calculate(label, is_multi, df):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    kosong = 0
    
    for index in range(0, len(df)):
        labels = df['label'][index].split('|')
        
        if type(df['pred_multiple'][index]) != float:
            preds = df['pred_multiple'][index].split('|')

            if label in labels and label in preds:
                tp += 1
            elif label not in labels and label not in preds:
                tn += 1
            elif label not in labels and label in preds:
                fp += 1
            elif label in labels and label not in preds:
                fn += 1
        elif label in labels:
            fn += 1
        else:
            tn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * ((precision * recall) / (precision + recall))
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    print(tp, tn, fp, fn, precision, recall, f1, accuracy)
    return tp, tn, fp, fn, precision, recall, f1, accuracy

In [21]:
import pandas as pd

def run(tipe):
    df = pd.read_csv('lda2vec works/hasil_kategori_lda2vec'+ tipe +'.csv')
    sf = pd.DataFrame(columns=['TP', 'TN', 'FP', 'FN', 'precision', 'recall', 'f1', 'accuracy'])

    cat = ['AMBIENCE', 'FOOD', 'SERVICE', 'VALUE']
    for c in cat:
        a, b, c, d, e, f, g, h = calculate(c, True, df)
        sf = sf.append({'TP':a, 'TN':b, 'FP':c, 'FN':d, 'precision':e, 'recall':f, 'f1':g, 'accuracy': h}, ignore_index=True)
    sf.to_excel("lda2vec works/hasil"+ tipe +".xlsx")

In [23]:
run("-nice")

174 371 122 32 0.5878378378378378 0.8446601941747572 0.6932270916334662 0.7796852646638054
355 239 48 57 0.8808933002481389 0.8616504854368932 0.8711656441717791 0.8497854077253219
137 519 9 34 0.9383561643835616 0.8011695906432749 0.8643533123028391 0.9384835479256081
124 416 141 18 0.4679245283018868 0.8732394366197183 0.6093366093366093 0.7725321888412017


In [16]:
run("-nice-filtered")

175 408 85 31 0.6730769230769231 0.8495145631067961 0.7510729613733904 0.8340486409155937
353 239 48 59 0.8802992518703242 0.8567961165048543 0.8683886838868389 0.8469241773962805
132 519 9 39 0.9361702127659575 0.7719298245614035 0.8461538461538461 0.9313304721030042
129 469 88 13 0.5944700460829493 0.9084507042253521 0.7186629526462396 0.8555078683834049


In [24]:
run("-nice-noun")

186 355 138 20 0.5740740740740741 0.9029126213592233 0.7018867924528301 0.7739628040057225
372 237 50 40 0.8815165876777251 0.9029126213592233 0.8920863309352518 0.871244635193133
140 517 11 31 0.9271523178807947 0.8187134502923976 0.8695652173913042 0.9399141630901288
124 413 144 18 0.4626865671641791 0.8732394366197183 0.6048780487804878 0.7682403433476395


In [25]:
run("-nice-noun-filtered")

169 405 88 37 0.6575875486381323 0.8203883495145631 0.7300215982721382 0.821173104434907
340 245 42 72 0.8900523560209425 0.8252427184466019 0.8564231738035265 0.8369098712446352
133 520 8 38 0.9432624113475178 0.7777777777777778 0.8525641025641026 0.9341917024320457
123 459 98 19 0.5565610859728507 0.8661971830985915 0.6776859504132232 0.8326180257510729


In [19]:
run("-nice-noun-filtered")

181 400 93 25 0.6605839416058394 0.8786407766990292 0.7541666666666668 0.8311874105865522
364 236 51 48 0.8771084337349397 0.883495145631068 0.8802902055622733 0.8583690987124464
132 519 9 39 0.9361702127659575 0.7719298245614035 0.8461538461538461 0.9313304721030042
129 465 92 13 0.583710407239819 0.9084507042253521 0.7107438016528925 0.8497854077253219
